In [ ]:
from time import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

from src.utils import get_path_projeto

In [ ]:
# 1. Carregando os dados
dir_projeto = get_path_projeto()
path_csv = dir_projeto / "data/staged/dados_empilhados.csv"
config_csv = {"sep": "\t", "encoding": "utf-8"}

dataset = pd.read_csv(path_csv, **config_csv)

# 2. Selecionando apenas dados sobre a geração de energia eólica
wind_power_generation = dataset.loc[:, ["interval_start_local", "wind"]]
wind_power_generation.rename(
    columns={"interval_start_local": "date", "wind": "power_generation"}, inplace=True
)

# 3. Dados para treinar o modelo
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"


def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i : i + lookback]
        target = dataset[i + 1 : i + lookback + 1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


lookback = 1
X, y = create_dataset(
    dataset=wind_power_generation["power_generation"].values, lookback=lookback
)


# 4. Criando a rede neural
class ArquiteturaRedeNeural(nn.Module):
    def __init__(self, lookback) -> None:
        super(ArquiteturaRedeNeural, self).__init__()
        self.lstm = nn.LSTM(
            input_size=lookback, hidden_size=50, num_layers=2, batch_first=True
        )
        self.linear = nn.Linear(in_features=50, out_features=lookback)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x


# 5. Treinando o modelo
def rotina_treino(modelo, fn_perda, otimizador, data_loader, num_epochs):
    t0 = time()
    for epoch in range(num_epochs):
        modelo.train()
        for X_batch, y_batch in data_loader:
            y_pred = modelo(X_batch)
            loss = loss_fn(y_pred, y_batch)
            otimizador.zero_grad()
            loss.backward()
            otimizador.step()
        if (epoch + 1) % 10 != 0:
            continue
        modelo.eval()
        with torch.no_grad():
            y_pred = modelo(X)
            train_rmse = np.sqrt(loss_fn(y_pred, y))
        exec_time = time() - t0
        print(
            "Epoch %d: train RMSE %.4f, time %.4f s"
            % (epoch + 1, train_rmse, exec_time)
        )
        t0 = time()
    return modelo


rede_neural = ArquiteturaRedeNeural(lookback=lookback)
loss_fn = nn.MSELoss()
otimizador = Adam(params=rede_neural.parameters(), lr=0.05)
data_loader = DataLoader(TensorDataset(X, y), shuffle=True, batch_size=10_000)

num_epochs = 2_000

rede_neural = rotina_treino(
    modelo=rede_neural,
    fn_perda=loss_fn,
    otimizador=otimizador,
    data_loader=data_loader,
    num_epochs=num_epochs,
)

/var/folders/g4/6w_d4rjs17n_cdn1lnl_nnf00000gn/T/ipykernel_35367/4034210821.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


Epoch 10: train RMSE 2674.3618, time 258.6779 s


KeyboardInterrupt: 

# SUGESTÃO GPT

In [ ]:

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

from src.utils import get_path_projeto

dir_projeto = get_path_projeto()

# 1. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load data and prepare it as usual
# (Assume dataset loading code here)


# 3. Modify the create_dataset function to use the device
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i : i + lookback]
        target = dataset[i + 1 : i + lookback + 1]
        X.append(feature)
        y.append(target)
    return (
        torch.tensor(X, dtype=torch.float32).to(device),
        torch.tensor(y, dtype=torch.float32).to(device),
    )


lookback = 1
X, y = create_dataset(
    dataset=wind_power_generation["power_generation"].values, lookback=lookback
)


# 4. Update the model to use the device
class ArquiteturaRedeNeural(nn.Module):
    def __init__(self, lookback) -> None:
        super(ArquiteturaRedeNeural, self).__init__()
        self.lstm = nn.LSTM(
            input_size=lookback, hidden_size=50, num_layers=2, batch_first=True
        )
        self.linear = nn.Linear(in_features=50, out_features=lookback)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x


# Move model to the device
rede_neural = ArquiteturaRedeNeural(lookback=lookback).to(device)
loss_fn = nn.MSELoss()
otimizador = Adam(params=rede_neural.parameters(), lr=0.05)

# 5. Update the data loader
data_loader = DataLoader(TensorDataset(X, y), shuffle=True, batch_size=10_000)


# 6. Train the model with the routine updated for the device
def rotina_treino(modelo, fn_perda, otimizador, data_loader, num_epochs):
    t0 = time()
    for epoch in range(num_epochs):
        modelo.train()
        for X_batch, y_batch in data_loader:
            # Move batches to device
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Forward pass
            y_pred = modelo(X_batch)
            loss = fn_perda(y_pred, y_batch)

            # Backward pass and optimization
            otimizador.zero_grad()
            loss.backward()
            otimizador.step()

        # Evaluation step every 10 epochs
        if (epoch + 1) % 10 == 0:
            modelo.eval()
            with torch.no_grad():
                y_pred = modelo(X)
                train_rmse = np.sqrt(fn_perda(y_pred, y).item())
            exec_time = time() - t0
            print(
                f"Epoch {epoch + 1}: train RMSE {train_rmse:.4f}, time {exec_time:.4f} s"
            )
            t0 = time()

    return modelo


# Start training
num_epochs = 2000
rede_neural = rotina_treino(
    modelo=rede_neural,
    fn_perda=loss_fn,
    otimizador=otimizador,
    data_loader=data_loader,
    num_epochs=num_epochs,
)